In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from scipy import sparse
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain

from model.data_parser import convert_jaad_dict_to_df, get_data

In [2]:
#Load data
data_dir = "C:/Users/max00/Documents/PoseRecognition/pedestrian-pose-recognition/data/JAAD_JSON_Labels/"

X, Y = convert_jaad_dict_to_df(get_data(data_dir))

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
sp_X_train = sparse.lil_matrix(X_train.values)
sp_Y_train = sparse.lil_matrix(Y_train.values)
sp_X_test = sparse.csr_matrix(X_test.values)
sp_Y_test = sparse.csr_matrix(Y_test.values)

Starting to read JAAD json files!


# Random forest

In [13]:
classifier = ClassifierChain(
        classifier=RandomForestClassifier(n_estimators=100),
        require_dense=[False, True]
    )

classifier.fit(sp_X_train, sp_Y_train)
# predict
predictions = classifier.predict(sp_X_test)

print(metrics.hamming_loss(sp_Y_test, predictions))
print(metrics.accuracy_score(sp_Y_test, predictions))

0.04928611760909898
0.8362507058159232


In [16]:
classifier.fit(X_train, Y_train)
predictions = classifier.predict(X_test)
df = pd.DataFrame.sparse.from_spmatrix(predictions)

0.04839880616278132
0.8384286520932483


TypeError: len() of unsized object

In [28]:
print(metrics.hamming_loss(Y_test, df))
print(metrics.accuracy_score(Y_test, df))
i=0
for col in df:
    df = df.rename(columns={col: str(int(col))})
for col in Y_test.columns:
    print(col + " accury:")
    print( metrics.accuracy_score(Y_test[col], df[str(i)]) )
    i += 1

0.04839880616278132
0.8384286520932483
look accury:
0.9244171977091232
action accury:
0.9409534564814068
cross accury:
0.8945712672420747
hand_gesture accury:
0.9983867064612406
nod accury:
0.9996773412922482


# BinaryRelevance

In [ ]:
clf = BinaryRelevance(
        classifier=SVC(),
        require_dense=[False, True]
    )

clf.fit(sp_X_train, sp_Y_train)
prediction = clf.predict(sp_X_test)


print(metrics.hamming_loss(sp_Y_test, prediction))
print(metrics.accuracy_score(sp_Y_test, prediction))

# LSTM

In [3]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense,Input

In [ ]:
features, coordinate_values = X_train.shape
model = Sequential()
model.add(LSTM(64, input_shape=(coordinate_values,1)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(5))  # output layer，units is the unit number for output

# compile model
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# train model
model.fit(X_train, Y_train, epochs=11, batch_size=32)

# prediction
predictions = model.predict(X_test)


print(predictions)
print(type(predictions))
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test, batch_size=128)
print(dict(zip(model.metrics_names, results)))

Epoch 1/11
1550/1550 [==============================] - 51s 30ms/step - loss: 0.2092 - accuracy: 0.2663
Epoch 2/11
1550/1550 [==============================] - 46s 30ms/step - loss: 0.1069 - accuracy: 0.7268
Epoch 3/11
 179/1550 [==>...........................] - ETA: 41s - loss: 0.1079 - accuracy: 0.7151